In [1]:
import requests

In [2]:
country = 'spain'

In [3]:
url = f"https://www.worldometers.info/coronavirus/country/{country}/"

In [4]:
req = requests.get(url)

In [5]:
req

<Response [200]>

In [6]:
from lxml.html import fromstring

In [7]:
root = fromstring(req.text)

In [8]:
scripts = root.xpath(".//script")

In [9]:
import re

In [10]:
lines = re.compile(r'[\n\t]')

In [11]:
comments = re.compile(r'\/\*(.*?)\*\/')

In [12]:
spaces = re.compile(r'[ ]{2,}')

In [13]:
chart = re.compile(r'(Highcharts\.chart\(\"[a-zA-z\-]*\"\, )(.*?)(\)\;)')

In [14]:
chart_title = re.compile(r'(Highcharts\.chart\(\"[a-zA-z\-]*\"\, )')

In [15]:
chart_end = re.compile(r'\)\;')

In [16]:
options = re.compile(r' [\w]+\: ')

In [17]:
xaxis = re.compile(r'(\"xAxis\"\: \{)(.*?)(\}\,)')

In [18]:
yaxis = re.compile(r'(\"yAxis\"\: \{ \"title\"\: \{ \"text\"\: \")(.*?)(\")')

In [19]:
series = re.compile(r'(\"series\"\: \[)(.*?)(\]\,)')

In [20]:
import json

In [21]:
import pandas as pd

In [22]:
from datetime import datetime

In [23]:
for script in scripts:
    if script.text_content().strip().__contains__("Highcharts.chart"):
        value = script.text_content().strip()
        
        value = lines.sub('', value)
        value = comments.sub('', value)
        value = spaces.sub(' ', value)
        value = value.replace('\'', '\"')
        value = chart.search(value).group(0)
        
        title = chart_title.search(value).group(0).replace('Highcharts.chart(\"', '').replace('\", ', '')
        
        print(title)
        
        value = chart_title.sub('', value)
        value = chart_end.sub('', value)
        
        words = options.findall(value)
        words = list(set(words))
        
        for word in words:
            value = value.replace(word, ' \"' + word.replace(' ', '').replace(':', '') + '\": ')
            
        column = yaxis.search(value).group(0).replace('"yAxis": { "title": { "text": "', '').replace('"', '')

        value = '{' + ', '.join([
            xaxis.search(value).group(0).replace('},', '}'),
            series.search(value).group(0).replace('],', ']'),
        ]) + '}'
        
        value = json.loads(value)
        
        x = value['series'][0]['data']
        
        y = [datetime.strptime(val + ', 2020', '%b %d, %Y') for val in value['xAxis']['categories']]
        
        data = pd.DataFrame({'Date': y, column: x})
        data.set_index('Date', inplace=True)
        data.to_csv(title + '.csv')

total-currently-infected-linear
deaths-cured-outcome-small
coronavirus-cases-linear
graph-cases-daily
graph-active-cases-total
coronavirus-deaths-linear
graph-deaths-daily
cases-cured-daily
deaths-cured-outcome
